<a href="https://colab.research.google.com/github/Madelinelai/Kaggle/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Madelinelai/Kaggle/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Madelinelai/Kaggle/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
test_df

In [ ]:
#資料預處理－缺失值填補
#step1　欄位分類　Cabin(船頭)，Embarked
#step2  PClass.Name(中間名), Sex
#step3  Age,SibSp,Parch,Ticket(算出同行有多少人), Fare

In [ ]:
data = pd.concat([train_df, test_df], ignore_index=True)
data = data.drop(["PassengerId", "Survived"], axis=1)

In [ ]:
# 此行是檢查資料裡是不是有N/A
na = data.isna().sum()
# Series[帶入根你的資料筆數一樣多True/False list]
na[na > 0].sort_values(ascending=False)

In [ ]:
#補上最常出現的類別/補上中位數
#Cabin有空值，use apply
#觀念說明如下
#a = pd.Series([1,2,3])
#def func(n):
#  return n

In [ ]:
#如果不是空值　就回傳 s / 是空值就會回傳N/A or None
def cabin_head(s):
    if not pd.isna(s):
        return s[0]
data["Cabin"] = data["Cabin"].apply(cabin_head)

In [ ]:
#算出同行有多少人，換句話說有多少人一起分享同張票
dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [ ]:
#補缺失值，補最常出現（類別Embarked：最常出現缺失值
#如果一堆測試資料，不要重算，直接補s
most = data['Embarked'].value_counts().idxmax()
data['Embarked'] = data['Embarked'].fillna(most)
na = data.isna().sum()
#Series(帶入，根據你的資料筆數一樣多True/False list)
na[na>0].sort_values(ascending=False)

In [ ]:
# 補缺失值(數值: 中位數)
med = data.median().drop(["Pclass"])
data = data.fillna(med)
na = data.isna().sum()
# Series[帶入根你的資料筆數一樣多True/False list]
na[na > 0].sort_values(ascending=False)

In [ ]:
#錄製影片有詳細解說

def name_convert(s):
    s = s.split(",")[-1].split(".")[0]
    s = s.strip()
    return s
counts = data["Name"].apply(name_convert).value_counts()
whitelist = counts[counts > 50].index
def name_convert(s):
    s = s.split(",")[-1].split(".")[0]
    s = s.strip()
    if s in whitelist:
        return s
    else:
        return None
data["Name"] = data["Name"].apply(name_convert)

In [ ]:
#再次檢視資料中其它欄位是否有需要ETL
#PClass屬大小類別123,Sex屬二值型，略過不必做
#整理完之後為２５個欄位，如下：
data = pd.get_dummies(data)
data = pd.get_dummies(data, columns=["Pclass"])
data

In [ ]:
#新增一個family欄位，故合併 SibSp+Parch（兄弟姐妹父母）
#以上是預處理的欄位嘗試能提昇Model正確率,千萬不要任意的刪除資料中的欄位
data["Family"] = data["SibSp"] + data["Parch"]
data
#整理完之後為２６個欄位，如下：